In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
!pip -q install -U bitsandbytes peft

In [4]:
!pip -q install --upgrade huggingface_hub

In [5]:
from huggingface_hub import login
login()

In [6]:
# meta-llama/Llama-3.2-1B-Instruct
from transformers import AutoModelForCausalLM, BitsAndBytesConfig,AutoTokenizer
import torch
# Load PHI 2 model with 4-bit quantization for efficient fine-tuning
# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_quant_type="nf4",
# #     bnb_4bit_compute_dtype=torch.float16
# )
model_name = "meta-llama/Llama-3.2-1B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name, 
#     quantization_config=bnb_config,
    torch_dtype = torch.float32,
    trust_remote_code=True
)


config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [8]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [10]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [7]:
select_feature = 'patch'
def feature_select(image_forward_outs):
    image_features = image_forward_outs.hidden_states[-1]
    if select_feature == 'patch':
        image_features = image_features[:, 1:]  # Skip CLS token if selecting patch
    elif select_feature == 'cls_patch':
        image_features = image_features  # Keep CLS + patch tokens
    else:
        raise ValueError(f'Unexpected select feature: {select_feature}')
    return image_features

In [11]:
import torch.nn as nn
import random
class MLPProjection(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dim=768, depth=2):
        super(MLPProjection, self).__init__()
        modules = []
        modules.append(nn.Linear(input_dim, hidden_dim,bias = False))
        
        for _ in range(1, depth):
            modules.append(nn.GELU())
            modules.append(nn.Linear(hidden_dim, output_dim,bias=False))
        
        self.mlp = nn.Sequential(*modules)
        
    
    def forward(self, x):
        return self.mlp(x)

class PHI2WithMLP(nn.Module):
    def __init__(self, phi2_model, mlp_projection):
        super(PHI2WithMLP, self).__init__()
        self.phi2_model = phi2_model
        self.mlp_projection = mlp_projection
        self.config = phi2_model.config
        
    def forward(self, image_embeddings=None,
                inputs_embeds=None,
                input_ids=None,
                attention_mask=None,
                labels=None,
                output_attentions=False, 
        output_hidden_states=False, 
        **kwargs):  # Catch any additional arguments):
        
        if input_ids is not None:
            token_embeddings = self.phi2_model.get_input_embeddings()(input_ids)
        elif inputs_embeds is not None:
            token_embeddings = inputs_embeds
        else:
            raise ValueError("You must provide either input_ids or inputs_embeds.")

        
        if image_embeddings is not None:
            # Apply MLP to image embeddings to map to text embedding space
            projected_image_embeddings = self.mlp_projection(image_embeddings).to(device = token_embeddings.device)
            
            # Get the sequence length for the image embeddings
            image_embedding_length = projected_image_embeddings.size(1)
            
            batch_size, text_sequence_length = attention_mask.shape
            
            # Extend attention mask for image embeddings (ones for image embedding positions)
            new_attention_mask = torch.cat(
                [torch.ones((batch_size,image_embedding_length), device=attention_mask.device),attention_mask ], dim=1
            )
            
            # Combine image and token embeddings
            combined_embeddings = torch.cat([projected_image_embeddings, token_embeddings], dim=1)  # Concatenating along sequence length
            
        else:
            # No image embeddings: Use only token embeddings and the original attention mask
            combined_embeddings = token_embeddings
            new_attention_mask = attention_mask
        if labels is not None:
            # Labels should match the sequence length of combined embeddings
            # If labels correspond only to text tokens, pad them to match the new sequence length
            if image_embeddings is not None:
                label_padding = torch.full(
                    (batch_size, image_embedding_length), -100, device=labels.device  # Use -100 for ignore index
                )
                new_labels = torch.cat([label_padding,labels], dim=1)
            else:
                new_labels = labels
        else:
            new_labels = labels
        # Pass the combined embeddings through the PHI2 model with the (updated or original) attention mask
        outputs = self.phi2_model(inputs_embeds=combined_embeddings, attention_mask=new_attention_mask,labels = new_labels, output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            **kwargs)

        return outputs
    
    def prepare_inputs_for_generation(self, input_ids, attention_mask=None, image_embeddings=None, **kwargs):
        # Generate inputs with projections where necessary
        if image_embeddings is not None:
            projected_image_embeddings = self.mlp_projection(image_embeddings)
            projected_image_embeddings = projected_image_embeddings.unsqueeze(0)
            
            token_embeddings = self.phi2_model.get_input_embeddings()(input_ids)
           
            combined_embeddings = torch.cat([projected_image_embeddings, token_embeddings], dim=1)
            image_embedding_length = projected_image_embeddings.size(1)
            
            image_embedding_length = projected_image_embeddings.size(1)
            
            batch_size, text_sequence_length = attention_mask.shape
            
            # Extend attention mask for image embeddings (ones for image embedding positions)
            new_attention_mask = torch.cat(
                [torch.ones((batch_size,image_embedding_length), device=attention_mask.device),attention_mask ], dim=1
            )
            
           
        else:
            combined_embeddings = self.phi2_model.get_input_embeddings()(input_ids)
            new_attention_mask = attention_mask

        return {
            "inputs_embeds": combined_embeddings,
            "attention_mask": new_attention_mask,
            
            **kwargs
        }
    
    def generate(self, input_ids, attention_mask=None, image_embeddings=None, **kwargs):
        self.eval()  # Set to evaluation mode
        # Prepare inputs for generation
        inputs = self.prepare_inputs_for_generation(input_ids, attention_mask, image_embeddings, **kwargs)
        # Use the model's built-in generate method
        return self.phi2_model.generate(**inputs)

def create_phi2_model_with_lora(mlp_projection,lan_model):
    
    for param in mlp_projection.parameters():
        param.requires_grad = True

    # Return PHI2 model with MLP projection
    return PHI2WithMLP(lan_model, mlp_projection)
    
model_embedding_dim = model.config.hidden_size  # This might change based on your model architecture

# Example usage
input_dim = 768  # Input dimension of image embeddings
output_dim = model_embedding_dim  # Target dimension of text embeddings
hidden_dim = 1024  # Hidden layer dimension of the MLP

mlp_projection = MLPProjection(input_dim, output_dim, hidden_dim, depth=2).to(device)  # Customize MLP
combined_model = create_phi2_model_with_lora(mlp_projection, model)


In [12]:
from peft import PeftModel, PeftConfig
peft_model_id = "Kartheekb7/results1"
loaded_model = PeftModel.from_pretrained(combined_model, peft_model_id)


adapter_config.json:   0%|          | 0.00/655 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/6.83M [00:00<?, ?B/s]

In [15]:
loaded_mlp_weights = torch.load("/kaggle/input/projection-weights/mlp_projection_weights.pth",map_location=torch.device('cpu'))

/tmp/ipykernel_30/2731402148.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_mlp_weights = torch.load("/kaggle/input/projection-weights/mlp_projection_weights.pth

In [16]:
loaded_model.base_model.model.mlp_projection.load_state_dict(loaded_mlp_weights)
print("Projection layer weights loaded successfully.")

Projection layer weights loaded successfully.


In [17]:
from transformers import GenerationConfig

# Create a new GenerationConfig with desired settings
generation_config = GenerationConfig(max_new_tokens=128, temperature=0.01, top_p=1)
loaded_model.generation_config = generation_config

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:601: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.01` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [20]:
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
# Load CLIP model and processor
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [21]:
def get_clip_embedding(image_path):
    image = Image.open(image_path)
    inputs = processor(images=image, return_tensors="pt", padding=True)
    with torch.no_grad():
        image_features = clip_model.get_image_features(**inputs)
        
        image_forward_outs = clip_model.vision_model(**inputs, output_hidden_states=True)
        image_features = feature_select(image_forward_outs)
        image_embedding = image_features.squeeze(0)
    return image_embedding

In [26]:
import torch


# Example input data
image_path = "/kaggle/input/sample/000000000009.jpg"
text_input = "What types of food can be seen in the image?"  # Example text input

# Prepare inputs
image_embedding = get_clip_embedding(image_path)

# Tokenize text input
input_encoding = tokenizer(
    text_input,
    return_tensors='pt',
    padding='max_length',
    truncation=True,
    max_length=256-49  # Set this to match your model's input size
)

image_embedding = image_embedding.squeeze(0).to(device)  # Shape: [embedding_dim]


In [27]:

outputs = loaded_model.generate(**input_encoding,image_embeddings = image_embedding, max_new_tokens=128, temperature=0.01, top_p=1)
# Decode output to text
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("Model response:", response)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Model response: 
The image shows a variety of food items, including a salad, a bowl of rice, a plate of meat, and a bowl of vegetables. There are also some fruits visible in the image. The food items are arranged in a colorful and visually appealing way, making it easy to identify and appreciate the different types of food. The salad and rice are likely to be a healthy and balanced meal option, while the meat and vegetables provide protein and essential nutrients. The fruits add natural sweetness and freshness to the dish. Overall, the image showcases a diverse and appetizing selection of food items.


In [43]:
import torchaudio
import torch
from transformers import WhisperProcessor, WhisperForConditionalGeneration
import ipywidgets as widgets
from IPython.display import display, Audio

# Load the Whisper model and processor
audio_processor = WhisperProcessor.from_pretrained("openai/whisper-small")
audio_model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")
audio_model.config.forced_decoder_ids = None



In [44]:
from transformers import pipeline
pipe = pipeline(
  "automatic-speech-recognition",
  model="openai/whisper-small",
  chunk_length_s=30,
  device=device,
)

In [41]:
result = pipe("/kaggle/working/temp_audio.wav",generate_kwargs={"language": "english"})


/opt/conda/lib/python3.10/site-packages/transformers/models/whisper/generation_whisper.py:496: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
You have passed language=english, but also have set `forced_decoder_ids` to [[1, None], [2, 50359]] which creates a conflict. `forced_decoder_ids` will be ignored in favor of language=english.


In [42]:
result['text']

" Hi, I'm going to miscarry."